In [28]:
import pandas as pd
from pandas import json_normalize
import time
import requests, bs4
import json
from bs4 import BeautifulSoup
import pygsheets
import gspread
from oauth2client.service_account import ServiceAccountCredentials as SAC
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [29]:
url = "https://nidss.cdc.gov.tw/nndss/DiseaseMap?id=19CoV"
html = requests.get(url)
objSoup = bs4.BeautifulSoup(html.text, 'lxml')
print(objSoup)

<!DOCTYPE HTML>
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>傳染病統計資料查詢系統</title>
<link href="/favicon.ico" rel="shortcut icon"/>
<meta content="IE=8,9,edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<link href="https://nidss.cdc.gov.tw/" id="ApplicationRoot" rel="home"/>
<link href="/Scripts/jquery-ui-1.12.1/jquery-ui.min.css" rel="stylesheet" type="text/css"/>
<link href="/Content/bootstrap.min.css" rel="stylesheet" type="text/css"/>
<link href="/Content/bootstrap-datetimepicker4.17.47.min.css" rel="stylesheet" type="text/css"/>
<link href="/Content/fontawesome-free-5.11.2-web/css/all.css" rel="stylesheet" type="text/css"/>
<link href="/Scripts/owl-carousel2/owl.carousel.min.css" rel="stylesheet" type="text/css"/>
<link href="/Scripts/owl-carousel2/owl.theme.default.min.css" rel="stylesheet" type="text/css"/>
<link href

In [30]:
if objSoup !=None:
    item1 = objSoup.find(class_="map-chart").script
    item1 = str(item1)
print(item1)

<script>
                                            var hmJson = [];
                                            let appendStr = '';

                                                
                                                hmJson.push({"type":"","shapefile":"../Maps/Ch/new/全國-GeoJSON.geojson","series":[{"code":"新北市","value":5535},{"code":"台北市","value":3864},{"code":"桃園市","value":543},{"code":"苗栗縣","value":420},{"code":"彰化縣","value":242},{"code":"基隆市","value":241},{"code":"台中市","value":172},{"code":"宜蘭縣","value":92},{"code":"高雄市","value":62},{"code":"新竹縣","value":51},{"code":"花蓮縣","value":50},{"code":"台南市","value":39},{"code":"屏東縣","value":33},{"code":"新竹市","value":32},{"code":"南投縣","value":30},{"code":"台東縣","value":22},{"code":"雲林縣","value":19},{"code":"嘉義縣","value":17},{"code":"嘉義市","value":9},{"code":"澎湖縣","value":5},{"code":"連江縣","value":4},{"code":"金門縣","value":0}],"properties":"CITYNAME","div_id":"container1","drill_down":true,"table_id":"tableStatisticsAppend","table_h1"

In [31]:
items1 = "{" +item1[item1.index('.geojson",')+10:item1.index('],"properties"')]+"]}"
print(items1)

{"series":[{"code":"新北市","value":5535},{"code":"台北市","value":3864},{"code":"桃園市","value":543},{"code":"苗栗縣","value":420},{"code":"彰化縣","value":242},{"code":"基隆市","value":241},{"code":"台中市","value":172},{"code":"宜蘭縣","value":92},{"code":"高雄市","value":62},{"code":"新竹縣","value":51},{"code":"花蓮縣","value":50},{"code":"台南市","value":39},{"code":"屏東縣","value":33},{"code":"新竹市","value":32},{"code":"南投縣","value":30},{"code":"台東縣","value":22},{"code":"雲林縣","value":19},{"code":"嘉義縣","value":17},{"code":"嘉義市","value":9},{"code":"澎湖縣","value":5},{"code":"連江縣","value":4},{"code":"金門縣","value":0}]}


In [32]:
dict_item1 = json.loads(items1)
print(dict_item1)
type(dict_item1)

{'series': [{'code': '新北市', 'value': 5535}, {'code': '台北市', 'value': 3864}, {'code': '桃園市', 'value': 543}, {'code': '苗栗縣', 'value': 420}, {'code': '彰化縣', 'value': 242}, {'code': '基隆市', 'value': 241}, {'code': '台中市', 'value': 172}, {'code': '宜蘭縣', 'value': 92}, {'code': '高雄市', 'value': 62}, {'code': '新竹縣', 'value': 51}, {'code': '花蓮縣', 'value': 50}, {'code': '台南市', 'value': 39}, {'code': '屏東縣', 'value': 33}, {'code': '新竹市', 'value': 32}, {'code': '南投縣', 'value': 30}, {'code': '台東縣', 'value': 22}, {'code': '雲林縣', 'value': 19}, {'code': '嘉義縣', 'value': 17}, {'code': '嘉義市', 'value': 9}, {'code': '澎湖縣', 'value': 5}, {'code': '連江縣', 'value': 4}, {'code': '金門縣', 'value': 0}]}


dict

In [33]:
dict_item1['series']

[{'code': '新北市', 'value': 5535},
 {'code': '台北市', 'value': 3864},
 {'code': '桃園市', 'value': 543},
 {'code': '苗栗縣', 'value': 420},
 {'code': '彰化縣', 'value': 242},
 {'code': '基隆市', 'value': 241},
 {'code': '台中市', 'value': 172},
 {'code': '宜蘭縣', 'value': 92},
 {'code': '高雄市', 'value': 62},
 {'code': '新竹縣', 'value': 51},
 {'code': '花蓮縣', 'value': 50},
 {'code': '台南市', 'value': 39},
 {'code': '屏東縣', 'value': 33},
 {'code': '新竹市', 'value': 32},
 {'code': '南投縣', 'value': 30},
 {'code': '台東縣', 'value': 22},
 {'code': '雲林縣', 'value': 19},
 {'code': '嘉義縣', 'value': 17},
 {'code': '嘉義市', 'value': 9},
 {'code': '澎湖縣', 'value': 5},
 {'code': '連江縣', 'value': 4},
 {'code': '金門縣', 'value': 0}]

In [34]:
df = json_normalize(dict_item1['series']) #Results contain the required data
df.columns=['地區', '累積確診人數']
df.head()

,地區,累積確診人數
0,新北市,5535
1,台北市,3864
2,桃園市,543
3,苗栗縣,420
4,彰化縣,242


In [35]:
#儲存資料
import time 
timestr = time.strftime("%Y%m%d") 
df.to_csv('DATASHEETS_COVD19_2/COVID19_2_'+timestr+'.csv', encoding="utf_8_sig")
df.to_csv('DATASHEETS_COVD19_2/COVID19_2.csv', encoding="utf_8_sig")

In [36]:
#取得昨日資料
from datetime import timedelta, datetime
yesterday = datetime.today() + timedelta(-1)
timestr_yesterday = yesterday.strftime('%Y%m%d')
df2 = pd.read_csv('DATASHEETS_COVD19_2/COVID19_2_'+timestr_yesterday+'.csv', index_col=0, thousands=',')
df2.head()

,地區,累積確診人數
0,新北市,5405
1,台北市,3796
2,桃園市,534
3,苗栗縣,409
4,彰化縣,240


In [37]:
# assign merged key
df3 = pd.merge(df, df2, on='地區')
df3.head()

,地區,累積確診人數_x,累積確診人數_y
0,新北市,5535,5405
1,台北市,3864,3796
2,桃園市,543,534
3,苗栗縣,420,409
4,彰化縣,242,240


In [38]:
df3['新增確診人數'] = df3['累積確診人數_x'] - df3['累積確診人數_y']
df3.head()

,地區,累積確診人數_x,累積確診人數_y,新增確診人數
0,新北市,5535,5405,130
1,台北市,3864,3796,68
2,桃園市,543,534,9
3,苗栗縣,420,409,11
4,彰化縣,242,240,2


In [39]:
#儲存資料
import time 
timestr = time.strftime("%Y%m%d") 
df3.to_csv('DATASHEETS_COVD19_2/COVID19_2_new'+timestr+'.csv', encoding="utf_8_sig")
df3.to_csv('DATASHEETS_COVD19_2/COVID19_2_new.csv', encoding="utf_8_sig")

In [40]:
# google sheets authentication
Json = './covid19.json'
Url = ['https://spreadsheets.google.com/feeds']

In [41]:
# 連線到資料表
Connect = SAC.from_json_keyfile_name(Json, Url)
GoogleSheets = gspread.authorize(Connect)

In [42]:
# 開啟資料表及工作表
Sheet = GoogleSheets.open_by_key('12ba57CqIauuqcYV93hUxwjmU7Wk_CoSLFytEuiuS1IQ')
# Sheets = Sheet.sheet1

In [43]:
# 查看此 GoogleSheet 內 Sheet 清單
wks_list = Sheet.worksheets()
print(wks_list)

[<Worksheet '工作表1' id:0>]


In [44]:
#選取by名稱
worksheet = Sheet.sheet1
#寫入資料
set_with_dataframe(worksheet, df3)
print("寫入成功")

寫入成功
